In [1]:
import pandas as pd
df = pd.read_csv('data/test2015/cleaned.csv')
df.head()

,jgmc,统计日期,QQ号,电子邮件,证件号码,通讯地址,姓名,性别,年龄,民族,...,高端小区客户标记,潜在高端客户标记,客户贡献度,客户活跃度,客户渠道偏好,客户金融资产偏好,核心客户号,金融资产代码_107,金融资产代码_130,金融资产代码_170
0,营业部,2015/11/30,NaN,yd273@163.com,NaN,四川省成都市龙泉驿区龙泉镇文明 东街106号移动公司 [P.C.610100],王桂英,女,47.0,NaN,...,NaN,NaN,较低贡献,高活跃,手机银行偏好型,贷款偏好型,1510128264,0.0,1.0,0.0
1,锦江支行,2015/11/30,568039505,568039505@qq.com,NaN,中国四川省成都市成都市高升桥南街5号,曾淑英,女,63.0,NaN,...,NaN,NaN,较低贡献,中活跃,POS交易偏好型,活储偏好型,1282863419,0.0,0.0,0.0
2,九眼桥支行,2015/11/30,13666266686,13666266686@qq.com,NaN,蜀汉路4号8栋2单元7楼14号,喻梅,女,65.0,NaN,...,NaN,NaN,低贡献,低活跃,柜台偏好型,理财偏好型,1308325181,0.0,1.0,0.0
3,青羊支行,2015/11/30,NaN,NaN,NaN,中国四川省致民路33号,陈晓容,女,46.0,NaN,...,NaN,NaN,低贡献,低活跃,POS交易偏好型,定存偏好型,1519464583,0.0,0.0,0.0
4,新津支行,2015/11/30,1064108847,1064108847@qq.com,NaN,中国四川省成都市新津县安西镇新漕村1组,王莎,女,28.0,NaN,...,NaN,NaN,低贡献,较高活跃,手机银行偏好型,理财偏好型,1501590367,0.0,1.0,0.0


In [2]:
df.shape

(247, 43)

In [3]:
"""
/mainClean.py

when called from command line $JUSHAPATH/$: python mainClean.py AbsDatadir
This script do the following stuff:

1.
"""
import sys

from __init__ import confUser
from preporcess import Cleaner, CleanerAnny
from tools import progressPrinter
import pandas as pd
import numpy as np
import os

#inputDir = sys.argv[1]
#confUser['DATADIR'] = sys.argv[1]




print(confUser['DATADIR'])

confUser['DATADIR'] = 'data/test2015'#sys.argv[1]

maptableDir = 'config/mappingTable.csv'
usefulCols = ['性别', '年龄', '婚姻', '学历', '从属行业',
'理财抗风险等级', '客户层级', '新老客户标记', '五级分类', '小微客户类型', '消费类资产产品', '纯消费性微贷标记',
'纯质押贷款标记', '非储投资偏好', '高金融资产标记', '购买大额他行理财标记', '大额消费标记', '信用卡高还款标记',
'信用卡高端消费标记', '优质行业标记', '高代发额客户标记','潜在高端客户标记', '客户贡献度',
'客户活跃度', '客户渠道偏好', '客户金融资产偏好']


cleaner = Cleaner(confUser)

cleaner.startCleaning(maptableDir, usefulCols)

cleannerAnny = CleanerAnny(confUser, ['金融资产'], v=False)
cleaner.saveNlog()

dfYall = cleannerAnny.startCleaning(intresedCode= [107, 130, 170])
#dfYall.rename(columns={'核心客户号':'jgmc'},inplace=True)

dfYallcsv = os.path.join(confUser['DATADIR'], 'dfYall.csv')
dfYall.to_csv(dfYallcsv, index=False)
#cleaner.cleanedDf['核心客户号'] = cleaner.cleanedDf.index
cleaner.cleanedDf['核心客户号'] = cleaner.cleanedDf.index.values
cleanedDf = pd.merge(cleaner.cleanedDf, dfYall, on='核心客户号')
fnc = os.path.join(confUser['DATADIR'], 'cleaned.csv')
cleanedDf.rename(columns={'jgmc':'所属分行'}, inplace=True)
cleanedDf['基金'] = cleanedDf['金融资产代码_170'].map(lambda x: '否' if x <1 else '是')
cleanedDf['主动负债'] = cleanedDf['金融资产代码_107'].map(lambda x: '否' if x <1 else '是')
cleanedDf['保险'] = cleanedDf['金融资产代码_130'].map(lambda x: '否' if x <1 else '是')
cleanedDf.drop(['金融资产代码_170', '金融资产代码_107', '金融资产代码_130'], axis=1, inplace=True)
sensitive_cols = ['姓名', '证件号码', '通讯地址', 'QQ号', '电子邮件']
cleanedDf.drop(sensitive_cols, axis=1, inplace=True)
for col in sensitive_cols:
    cleanedDf[col] = [np.nan]*cleanedDf.shape[0]

cleanedDf.to_csv(fnc, index=False)




Using cmappertools v1.0.24.
data/tempdir
data/test2015/rawdata
<<<0.00>>>
<<<0.10>>>


b'Skipping line 1477: expected 39 fields, saw 40\nSkipping line 7510: expected 39 fields, saw 40\n'


Dataframes are ready!
<<<0.20>>>
Concate done!????
<<<0.30>>>
nan filled and df sub selected!
<<<0.40>>>
data mapping done!
<<<0.50>>>
data outlier treated!
<<<0.60>>>



data standardization done!
<<<0.70>>>
data/test2015/rawdata
['尝试标准化数据时，发现有非数字的列: 从属行业', '尝试标准化数据时，发现有非数字的列: 消费类资产产品', '尝试标准化数据时，发现有非数字的列: 非储投资偏好']
<<<0.80>>>
<<<0.90>>>
Files are generated!
<<<1.00>>>
0
(50168, 6)
(0, 1)


KeyError: '金融资产代码_170'

In [13]:
cleaner = Cleaner(confUser)

cleaner.startCleaning(maptableDir, usefulCols)

data/test2015/rawdata
<<<0.00>>>
<<<0.10>>>


b'Skipping line 1477: expected 39 fields, saw 40\nSkipping line 7510: expected 39 fields, saw 40\n'


Dataframes are ready!
<<<0.20>>>
Concate done!????
<<<0.30>>>
nan filled and df sub selected!
<<<0.40>>>
data mapping done!
<<<0.50>>>
data outlier treated!
<<<0.60>>>



data standardization done!
<<<0.70>>>


In [20]:
uidf =  pd.DataFrame(cleaner.tranDf.index.values, columns=['uid'])
uidf

,uid
0,1.518969e+09
1,1.317248e+09
2,1.189889e+09
3,1.510128e+09
4,1.506503e+09
5,1.282863e+09
6,1.293461e+09
7,1.201989e+09
8,1.293718e+09
9,1.504666e+09


In [19]:
cleaner.tranDf.index

Float64Index([1518968801.0, 1317247708.0, 1189888621.0, 1510128264.0,
              1506502590.0, 1282863419.0, 1293460765.0, 1201988638.0,
              1293717977.0, 1504665616.0,
              ...
              1252209544.0, 1501382532.0, 1506782398.0, 1242580883.0,
              1509634239.0, 1288051835.0, 1244413777.0, 1512794276.0,
              1510740433.0, 1508829177.0],
             dtype='float64', name='客户代码', length=9998)

In [9]:
cleannerAnny = CleanerAnny(confUser, ['金融资产'], v=False)
#dfYall = cleannerAnny.startCleaning(intresedCode= [107, 130, 170])

data/test2015/rawdata


In [12]:
fnuid = os.path.join(cleannerAnny.datadir, 'uid.csv')
uidArr = pd.read_csv(fnuid).values
uidArr.shape

(0, 1)

In [16]:
fnuid

'data/test2015/uid.csv'

In [17]:
cleaner.error

['尝试标准化数据时，发现有非数字的列: 从属行业',
 '尝试标准化数据时，发现有非数字的列: 消费类资产产品',
 '尝试标准化数据时，发现有非数字的列: 非储投资偏好']

In [19]:
for i,j,k in cleaner.tranDf[['从属行业', '消费类资产产品', '非储投资偏好']].values:
    try:
        float(i)
        float(j)
        float(k)
    except Exception as e:
        print(i, j, k)

科学研究、技术服务和地质勘探业 0 0
6.0 3.5 非储中贵金属偏好
0 商铺按揭,住房按揭 3.0
0 商铺按揭,住房按揭 0
4.5 0 非储中贵金属偏好
科学研究、技术服务和地质勘探业 0 0
科学研究、技术服务和地质勘探业 2.0 0
科学研究、技术服务和地质勘探业 2.0 0
6.0 商铺按揭,住房按揭 3.0
科学研究、技术服务和地质勘探业 0 0
0 3.0 非储中贵金属偏好
6.0 其他消费类,消费型微 3.0
0 0 非储中贵金属偏好
0 0 非储中贵金属偏好


In [29]:
for r, i in cleaner.tranDf.values:
    for 

[0.0 35.0 0.0 0.0 0 0.0 1.0 1.0 4.0 0.0 2.0 0.0 0.0 0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 2.0 1.0 1.4 4.0]


In [35]:
cleaner.tranDf.ix[:, [0,1]]

,性别,年龄
客户代码,,
1.518969e+09,0.0,5.0
1.317248e+09,0.0,49.0
1.189889e+09,0.0,37.0
1.510128e+09,0.0,47.0
1.506503e+09,0.0,29.0
1.282863e+09,0.0,63.0
1.293461e+09,0.0,46.0
1.201989e+09,1.0,37.0
1.293718e+09,0.0,24.0


In [5]:

cleaner2 = Cleaner(confUser)
dfs = cleaner2._getDfs(0)

dfs1 = cleaner2._getDfs(1)


data/test2015/rawdata


b'Skipping line 1477: expected 39 fields, saw 40\nSkipping line 7510: expected 39 fields, saw 40\n'


In [21]:
maptable = pd.read_csv('config/mappingTable.csv', header=None)

In [23]:
maptable = maptable.ix[:, :1].values

In [28]:
np.vstack((maptable, [2,3], [4,5]))

array([['女', 0.0],
       ['男', 1.0],
       ['已婚', 2.0],
       ['未婚', 1.0],
       ['其他', 0.0],
       ['离异', 3.0],
       ['中专或高中', 2.0],
       ['未知', 0.0],
       ['本科', 4.0],
       ['初中及以下', 1.0],
       ['研究生', 5.0],
       ['博士及以上', 6.0],
       ['专科', 3.0],
       ['公共管理与社会组织', 4.5],
       ['文化、体育、娱乐业', 3.8],
       ['电力、燃气及水的生产和供应业', 4.0],
       ['交通运输、仓储和邮政业', 3.5],
       ['农、林、牧、渔业', 1.0],
       ['批发和零售业', 5.0],
       ['其他', 0.0],
       ['制造业', 5.5],
       ['居民服务和其他服务业', 5.0],
       ['房地产业', 3.0],
       ['采矿业', 2.0],
       ['金融业', 6.0],
       ['信息传输、计算机服务和软件业', 6.5],
       ['建筑业', 2.5],
       ['租赁和商务服务业', 3.3],
       ['教育', 4.8],
       ['卫生、社会保障和社会福利业', 4.6],
       ['中庸型', 3.0],
       ['未知', 0.0],
       ['非常保守型', 1.0],
       ['温和成长型', 4.0],
       ['稳健保守型', 2.0],
       ['积极成长型', 5.0],
       ['无效户', 0.0],
       ['有效户', 1.0],
       ['私人', 5.0],
       ['金卡', 3.0],
       ['银卡', 2.0],
       ['钻卡', 4.0],
       ['老客户', 2.0],
       ['新客户', 1.0],
       

In [6]:
for i in dfs1:
    print(i.shape)

(2000, 2)
(2000, 2)


In [7]:
df = pd.concat(dfs)

In [8]:
df.shape

(9998, 39)

In [9]:
df

,统计日期,客户代码,QQ号,电子邮件,证件号码,通讯地址,姓名,性别,年龄,民族,...,信用卡高还款标记,信用卡高端消费标记,优质行业标记,高代发额客户标记,高端小区客户标记,潜在高端客户标记,客户贡献度,客户活跃度,客户渠道偏好,客户金融资产偏好
0,2015/11/30,1.518969e+09,NaN,1@qq.com,NaN,中国四川省黄金路成钞家园15216216,黄燕,女,35.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,中贡献,低活跃,POS交易偏好型,贷款偏好型
1,2015/11/30,1.317248e+09,NaN,1@QQ.COM,NaN,成华区小龙桥83号2－3－5,杨凤英,女,49.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,较高贡献,低活跃,POS交易偏好型,活储偏好型
2,2015/11/30,1.189889e+09,NaN,13982179720@139.com,NaN,四川省成都市武侯区武侯大道双丰 路５３＃军通大厦３０３成都易卡 网络技术有限公司销售部 ...,何瑛,女,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,低贡献,低活跃,自助设备偏好型,活储偏好型
3,2015/11/30,1.510128e+09,NaN,yd273@163.com,NaN,四川省成都市龙泉驿区龙泉镇文明 东街106号移动公司 [P.C.610100],王桂英,女,47.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,较低贡献,高活跃,手机银行偏好型,贷款偏好型
4,2015/11/30,1.506503e+09,420642312,420642312@qq.com,NaN,undefined,王丽梅,女,29.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015/11/30,1.282863e+09,568039505,568039505@qq.com,NaN,中国四川省成都市成都市高升桥南街5号,曾淑英,女,63.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,较低贡献,中活跃,POS交易偏好型,活储偏好型
6,2015/11/30,1.293461e+09,NaN,0@0.com,NaN,中国四川省成都市市辖区成都市锦江区华润路2号16栋1单元803,唐萍,女,46.0,NaN,...,NaN,NaN,NaN,NaN,NaN,潜在高端客户,高贡献,中活跃,手机银行偏好型,理财偏好型
7,2015/11/30,1.201989e+09,34754292,34754292@QQ.COM,NaN,四川省成都市 武侯区高升桥南街５号 １－２－２－１ [P.C.610041],熊锴,男,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015/11/30,1.293718e+09,NaN,NaN,NaN,四川省仁寿县珠嘉乡园山村2组,李清,女,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015/11/30,1.504666e+09,NaN,0@0.com,NaN,湖北省郧西县土门镇双庙村3组范家沟口27号,余祖春,男,44.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,高贡献,低活跃,网银偏好型,贷款偏好型


In [30]:
df.drop_duplicates(subset='客户代码', inplace= True)

In [31]:
df.shape

(9998, 39)

In [33]:
df1 = pd.concat(dfs1)

In [34]:
df1.shape

(4000, 2)

In [36]:
df1.drop_duplicates(subset='khdm', inplace= True)

In [37]:
df1.set_index('khdm', drop=True, inplace=True)

In [42]:
df.set_index('客户代码', drop=True, inplace=True)

In [43]:
dfmaster = pd.concat([df, df1], axis=1, join='inner')

In [44]:
dfmaster.shape

(247, 39)

In [15]:
paths = []
for dirpath, dirnames, filenames in os.walk(confUser['DATADIR']):
    for fn in filenames:
        if confUser["KEYWORD"][0] in fn and fn.endswith('.txt'):
            #fn has pattern: *keword*.txt*
            fnPath = os.path.join(dirpath, fn)
            paths.append(fnPath)

In [16]:
paths

['data/test2015/rawdata/正样本_客户画像.txt', 'data/test2015/rawdata/负样本_客户画像.txt']

In [19]:
df1 = pd.read_csv(paths[0], delimiter='|', encoding='gb18030')
df2 = pd.read_csv(paths[1], delimiter='|', encoding='gb18030', error_bad_lines=False)

b'Skipping line 1477: expected 39 fields, saw 40\nSkipping line 7510: expected 39 fields, saw 40\n'


In [21]:
df2

,统计日期,客户代码,QQ号,电子邮件,证件号码,通讯地址,姓名,性别,年龄,民族,...,信用卡高还款标记,信用卡高端消费标记,优质行业标记,高代发额客户标记,高端小区客户标记,潜在高端客户标记,客户贡献度,客户活跃度,客户渠道偏好,客户金融资产偏好
0,2015/11/30,1518968801,?,1@qq.com,未知,中国四川省黄金路成钞家园15216216,黄燕,女,35,其他,...,,,,,,,中贡献,低活跃,POS交易偏好型,贷款偏好型
1,2015/11/30,1317247708,?,1@QQ.COM,未知,成华区小龙桥83号2－3－5,杨凤英,女,49,其他,...,,,,,,,较高贡献,低活跃,POS交易偏好型,活储偏好型
2,2015/11/30,1189888621,?,13982179720@139.com,未知,四川省成都市武侯区武侯大道双丰 路５３＃军通大厦３０３成都易卡 网络技术有限公司销售部 ...,何瑛,女,37,其他,...,,,,,,,低贡献,低活跃,自助设备偏好型,活储偏好型
3,2015/11/30,1510128264,?,yd273@163.com,未知,四川省成都市龙泉驿区龙泉镇文明 东街106号移动公司 [P.C.610100],王桂英,女,47,其他,...,未知,未知,未知,未知,未知,未知,较低贡献,高活跃,手机银行偏好型,贷款偏好型
4,2015/11/30,1506502590,420642312,420642312@qq.com,未知,undefined,王丽梅,女,29,其他,...,,,,,,,未知,未知,未知,未知
5,2015/11/30,1282863419,568039505,568039505@qq.com,未知,中国四川省成都市成都市高升桥南街5号,曾淑英,女,63,其他,...,,,,,,,较低贡献,中活跃,POS交易偏好型,活储偏好型
6,2015/11/30,1293460765,?,0@0.com,未知,中国四川省成都市市辖区成都市锦江区华润路2号16栋1单元803,唐萍,女,46,其他,...,,,,,,潜在高端客户,高贡献,中活跃,手机银行偏好型,理财偏好型
7,2015/11/30,1201988638,34754292,34754292@QQ.COM,未知,四川省成都市 武侯区高升桥南街５号 １－２－２－１ [P.C.610041],熊锴,男,37,其他,...,,,,,,,未知,未知,未知,未知
8,2015/11/30,1293717977,?,NaN,未知,四川省仁寿县珠嘉乡园山村2组,李清,女,24,其他,...,,,,,,,未知,未知,未知,未知
9,2015/11/30,1504665616,?,0@0.com,未知,湖北省郧西县土门镇双庙村3组范家沟口27号,余祖春,男,44,其他,...,未知,未知,未知,未知,未知,未知,高贡献,低活跃,网银偏好型,贷款偏好型


In [10]:
cleaner.cleanedDf

,jgmc,统计日期,QQ号,电子邮件,证件号码,通讯地址,姓名,性别,年龄,民族,...,信用卡高端消费标记,优质行业标记,高代发额客户标记,高端小区客户标记,潜在高端客户标记,客户贡献度,客户活跃度,客户渠道偏好,客户金融资产偏好,核心客户号
1510128264,营业部,2015/11/30,NaN,yd273@163.com,NaN,四川省成都市龙泉驿区龙泉镇文明 东街106号移动公司 [P.C.610100],王桂英,女,47.0,NaN,...,NaN,NaN,NaN,NaN,NaN,较低贡献,高活跃,手机银行偏好型,贷款偏好型,1510128264
1282863419,锦江支行,2015/11/30,568039505,568039505@qq.com,NaN,中国四川省成都市成都市高升桥南街5号,曾淑英,女,63.0,NaN,...,NaN,NaN,NaN,NaN,NaN,较低贡献,中活跃,POS交易偏好型,活储偏好型,1282863419
1308325181,九眼桥支行,2015/11/30,13666266686,13666266686@qq.com,NaN,蜀汉路4号8栋2单元7楼14号,喻梅,女,65.0,NaN,...,NaN,NaN,NaN,NaN,NaN,低贡献,低活跃,柜台偏好型,理财偏好型,1308325181
1519464583,青羊支行,2015/11/30,NaN,NaN,NaN,中国四川省致民路33号,陈晓容,女,46.0,NaN,...,NaN,NaN,NaN,NaN,NaN,低贡献,低活跃,POS交易偏好型,定存偏好型,1519464583
1501590367,新津支行,2015/11/30,1064108847,1064108847@qq.com,NaN,中国四川省成都市新津县安西镇新漕村1组,王莎,女,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,低贡献,较高活跃,手机银行偏好型,理财偏好型,1501590367
1514620510,锦江支行,2015/11/30,751167644,751167644@qq.com,NaN,中国四川省成都市成都学院,文捷,女,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,较高贡献,低活跃,POS交易偏好型,定存偏好型,1514620510
1517374218,西体路支行,2015/11/30,396762766,396762766@qq.com,NaN,中国四川省成都市锦江区大业路,戴双凤,女,34.0,NaN,...,NaN,NaN,NaN,NaN,潜在高端客户,较高贡献,高活跃,手机银行偏好型,活储偏好型,1517374218
1507621646,建设路支行,2015/11/30,NaN,1@qq.com,NaN,成都市成华区地勘路1号4栋1单元5楼9号,林先碧,女,64.0,NaN,...,NaN,NaN,NaN,NaN,NaN,中贡献,较高活跃,渠道均衡型,资产均衡型,1507621646
1161915605,荷花池支行,2015/11/30,NaN,NaN,NaN,中国四川省成都市晋吉东二街236号,王学军,男,45.0,NaN,...,NaN,NaN,NaN,NaN,NaN,较高贡献,低活跃,POS交易偏好型,理财偏好型,1161915605
1244811778,锦江支行,2015/11/30,NaN,48@qq.om,NaN,成都市武侯区棕苑巷1号3单元6号,林华珍,女,65.0,NaN,...,NaN,NaN,NaN,NaN,NaN,高贡献,低活跃,POS交易偏好型,活储偏好型,1244811778


In [6]:
dfYall

,核心客户号,金融资产代码_107,金融资产代码_130,金融资产代码_170
0,1510128264,0.0,1.0,0.0
1,1282863419,0.0,0.0,0.0
2,1308325181,0.0,1.0,0.0
3,1519464583,0.0,0.0,0.0
4,1501590367,0.0,1.0,0.0
5,1514620510,0.0,0.0,0.0
6,1517374218,0.0,0.0,0.0
7,1507621646,0.0,1.0,1.0
8,1161915605,0.0,0.0,0.0
9,1244811778,0.0,0.0,0.0


In [29]:
cleaner.cleanedDf.index.values

array([1156224749, 1507158773, 1338742696, ..., 1510873791, 1234289690,
       1525530057])

In [2]:
from preporcess import CleanerAnny
from __init__ import confUser

In [6]:
confUser['DATADIR'] = 'data/tempdir/'
cleannerAnny = CleanerAnny(confUser, ['金融资产'], v=False)
dfYall = cleannerAnny.startCleaning(intresedCode= [107, 130, 170])

data/tempdir/rawdata
11596
(69481, 6)


/Users/xl-macbook/anaconda/lib/python2.7/site-packages/pandas/core/common.py:452: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask = arr == x
/Users/xl-macbook/anaconda/lib/python2.7/site-packages/pandas/core/common.py:464: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask |= arr == x


KeyError: '\xe9\x87\x91\xe8\x9e\x8d\xe8\xb5\x84\xe4\xba\xa7\xe4\xbb\xa3\xe7\xa0\x81'